##  <u>Movendo modelo através do IfcOpenShell (baseado em objeto de referência)</u>

In [32]:
import ifcopenshell
from ifcopenshell.util.selector import Selector

# Lendo os arquivos de origem (que será movido) e o arquivo destino (com a coordenada destino para o deslocamento)
ifcOrigin = ifcopenshell.open('ifc/lajes.ifc')
ifcTarget = ifcopenshell.open('ifc/pilares.ifc')

# Obtendo o objeto IfcSite de cada arquivo
siteOrigin = ifcOrigin.by_type('IfcSite')[0]
siteTarget = ifcTarget.by_type('IfcSite')[0]


### 1. Obtendo o marco

In [37]:
# Obtendo o objeto de referência de cada arquivo

selector = Selector()
marcoOrigin = selector.parse(ifcOrigin, '.IfcBuildingElementProxy[Name *= "marco"]')[0]
marcoTarget = selector.parse(ifcTarget, '.IfcBuildingElementProxy[Name *= "marco"]')[0]

In [38]:
marcoOrigin

#317=IfcBuildingElementProxy('2oqvjbeo17q91TiybToJGP',#42,'marco:marco:288474',$,'marco:marco',#316,#312,'288474',$)

### 2. Calcular a diferença entre os arquivos

<img src="img/ifclocalplacement.png" style="width:600px; height:400px"/>

In [40]:
# Obtendo as coordenadas de origem dos marcos de cada arquivo

xyzTarget = marcoTarget.ObjectPlacement.RelativePlacement.Location.Coordinates
xyzOrigin = marcoOrigin.ObjectPlacement.RelativePlacement.Location.Coordinates

# Obtendo a origem do arquivo a ser movido

localPlacement = siteOrigin.ObjectPlacement
xyzSite = localPlacement.RelativePlacement.Location.Coordinates
axisSite = localPlacement.RelativePlacement.Axis
refDir =  localPlacement.RelativePlacement.RefDirection

# Obtendo as diferenças

deltaX = xyzTarget[0] - xyzOrigin[0]
deltaY = xyzTarget[1] - xyzOrigin[1]
deltaZ = xyzTarget[2] - xyzOrigin[2]

# Calculando a nova origem

newXyz = (xyzOrigin[0] + deltaX, xyzOrigin[1] + deltaY, xyzOrigin[2] + deltaZ)

## 3. Criando um novo objeto de localização

<img src="img/newifclocalplacement.png" style="width:650px; height:450px"/>

In [ ]:

# Criando um novo IfcCartesianPoint e IfcAxis2Placement3D

newCartesianPoint = ifcOrigin.create_entity('IfcCartesianPoint', Coordinates = newXyz )
newAxis2Placement = ifcOrigin.create_entity('IfcAxis2Placement3D',
                                            Location=newCartesianPoint,
                                            Axis=axisSite,
                                            RefDirection=refDir)



### 3. Deslocando a origem do site e gravando um novo arquivo

In [41]:
# Movendo o arquivo origem
localPlacement.RelativePlacement = newAxis2Placement

# Gravando um novo arquivo
ifcOrigin.write('ifc/newLajes.ifc')